# Imports

In [1]:
exec(open('setup.py').read())

from notebooks import *

2025-09-18 18:12:02 | notebooks | INFO | 🔑 Environment loaded from: /home/klaus/Projects/car-system-agentic-ai/.env
2025-09-18 18:12:02 | notebooks | INFO | 🔑 GOOGLE_API_KEY: ✅ Found


# Definitions

In [ ]:
gemini_model_1 = models.GeminiModel(model="gemini-2.5-flash", prompt="You are a helpful assistant.")
gemini_model_2 = models.GeminiModel(model="gemini-2.5-flash", prompt="You are a helpful assistant.")

node_1 = nodes.AgenticNode(
    routing_options=["node_2"],
    model=gemini_model_1,
    tools=[],
)

node_2 = nodes.AgenticNode(
    routing_options=[],
    model=gemini_model_2,
    tools=[],
)

# Tests